# A Basic Retrieval Augmented Generation (RAG) flow with the Kenya 2010 Constitution

The goal of this is to be able to ask questions and receive responses about information contained in articles of the Kenya 2010 Constitution. This is useful for getting quick responses without having to manually search the entire document.

## Pre-processing

We have a JSON copy of the constitution's articles extracted from a PDF version of the constitution in [this][0] Github repo. We fetch the JSON:

[0]: https://github.com/programmer-ke/constitution_kenya

In [1]:
!wget https://raw.githubusercontent.com/programmer-ke/constitution_kenya/refs/heads/master/json/ConstitutionKenya2010.json -O constitution.json

--2024-10-02 15:33:29--  https://raw.githubusercontent.com/programmer-ke/constitution_kenya/refs/heads/master/json/ConstitutionKenya2010.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 304441 (297K) [text/plain]
Saving to: ‘constitution.json’

constitution.json   100%[===================>] 297.31K   991KB/s    in 0.3s    

2024-10-02 15:33:31 (991 KB/s) - ‘constitution.json’ saved [304441/304441]



In [2]:
import json

In [3]:
with open('constitution.json', 'rt') as f:
    articles = json.load(f)

In [4]:
articles[64:67]

[{'number': 65,
  'title': 'Landholding by non-citizens.',
  'lines': ['(1)  A person who is not a citizen may hold land on the basis of leasehold\n',
   'tenure only, and any such lease, however granted, shall not exceed ninety-nine years.\n',
   '(2)  If a provision of any agreement, deed, conveyance or document of whatever\n',
   'nature purports to confer on a person who is not a citizen an interest in land greater\n',
   'than a ninety-nine year lease, the provision shall be regarded as conferring on the\n',
   'person a ninety-nine year leasehold interest, and no more.\n',
   '(3)  For purposes of this Article —\n',
   '(a) a body corporate shall be regarded as a citizen only if the body corporate\n',
   'is wholly owned by one or more citizens; and\n',
   '(b) property held in trust shall be regarded as being held by a citizen only\n',
   'if all of the beneficial interest of the trust is held by persons who are\n',
   'citizens.\n',
   '(4)  Parliament may enact legislation to 

It is a list of the articles contained in the constitution with the associated article number, the chapter it belongs to, if the chapter is divided into parts, we have the part number and title, and finally the lines that compose the article's clauses.

We do some processing to make searching easier:
- combine the article lines into a single text field
- have a single text field for each of chapter, part and article title
- Where part doesn't exist, have it as a zero length string

In [5]:
search_fields = []
for article in articles:
    
    article_text = "".join(article['lines'])
    article_title = f"Article {article['number']}: {article['title']}"
    chapter_number, chapter_title = article['chapter']
    chapter_text = f"Chapter {chapter_number}: {chapter_title}"
    part_text = ""
    
    if article['part']:
        part_num, part_title = article['part']
        part_text = f'Part {part_num}: {part_title}'
        
    search_fields.append({
        "article_title": article_title,
        "article_text": article_text,
        "chapter": chapter_text,
        "part": part_text
    })
    

In [6]:
search_fields[:3]

[{'article_title': 'Article 1: Sovereignty of the people.',
  'article_text': '(1)  All sovereign power belongs to the people of Kenya and shall be exercised\nonly in accordance with this Constitution.\n(2)  The people may exercise their sovereign power either directly or through their\ndemocratically elected representatives.\n(3)  Sovereign power under this Constitution is delegated to the following State\norgans, which shall perform their functions in accordance with this Constitution—\n(a) Parliament and the legislative assemblies in the county governments;\n(b) the national executive and the executive structures in the county\ngovernments; and\n(c) the Judiciary and independent tribunals.\n(4)  The sovereign power of the people is exercised at—\n(a) the national level; and\n(b) the county level.\n',
  'chapter': 'Chapter 1: SOVEREIGNTY OF THE PEOPLE AND SUPREMACY OF THIS CONSTITUTION',
  'part': ''},
 {'article_title': 'Article 2: Supremacy of this Constitution.',
  'article_text':

In [7]:
search_fields[64:67]

[{'article_title': 'Article 65: Landholding by non-citizens.',
  'article_text': '(1)  A person who is not a citizen may hold land on the basis of leasehold\ntenure only, and any such lease, however granted, shall not exceed ninety-nine years.\n(2)  If a provision of any agreement, deed, conveyance or document of whatever\nnature purports to confer on a person who is not a citizen an interest in land greater\nthan a ninety-nine year lease, the provision shall be regarded as conferring on the\nperson a ninety-nine year leasehold interest, and no more.\n(3)  For purposes of this Article —\n(a) a body corporate shall be regarded as a citizen only if the body corporate\nis wholly owned by one or more citizens; and\n(b) property held in trust shall be regarded as being held by a citizen only\nif all of the beneficial interest of the trust is held by persons who are\ncitizens.\n(4)  Parliament may enact legislation to make further provision for the operation\nof this Article.\n',
  'chapter'

## Retrieval